<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/idash_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 무조건 해야하는 부분 1
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [849 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [834 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [ ]:
## 무조건 해야하는 부분 2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
## 번역문장 크롤링
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import os
import pandas as pd
from tqdm import tqdm

driver.implicitly_wait(10)
driver.get("https://papago.naver.com/")

# file_list = os.listdir('/content/drive/MyDrive/KB/pre')

text = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="txtSource"]')))
left = ''
cur = ''
# for fi in file_list[1:]:
# for fi in ['재정ㆍ경제일반_pre.csv']:
for fi in ['/content/상업ㆍ무역ㆍ공업_pre.csv']:
  print(fi)
  trans = []
  # df = pd.read_csv(f'/content/drive/MyDrive/KB/pre/{fi}', index_col=0)
  df = pd.read_csv(f'{fi}', index_col=0)
  df = df[df['name'] == '선불식 할부거래에서의 소비자보호 지침\n        이 영문법령은 공정거래위원회에서 제공하였습니다']

  for row in tqdm(df.values):
    text.send_keys(row[1])
    time.sleep(2)

    translated = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]')))
    cur = translated.text
    # if left == cur:
    #   print('크롤링 속도가 너무 빠릅니다.')
    #   time.sleep(0.3)
    #   translated = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[3]/c-wiz[2]/div/div[9]/div/div[1]')))
    #   cur = translated.text

    trans.append(cur)
    left = cur
    text.clear()

  df['trans'] = trans
  # df.to_csv(f'/content/drive/MyDrive/KB/trans/{fi[:-4]}_trans.csv', index=False)

driver.close()

/content/상업ㆍ무역ㆍ공업_pre.csv


100%|██████████| 41/41 [03:01<00:00,  4.43s/it]


In [ ]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")

laws_per_cat = {}
# for i in range(2, 47):
for i in range(2, 47):
  category = driver.find_element(By.CSS_SELECTOR, f"#tree_{i}_span") # 카테고리 선택
  cat = category.text
  laws_per_cat[cat.split()[1]] = int(cat.split('(')[-1].replace(')',''))

# category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]

cur_sum_pages = 1
total_law = 1
from collections import defaultdict
boryu = defaultdict(list)

while total_law <= 2341:
  for c in category_list:
    cur_law = 1

    driver.switch_to.default_content()
    category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, c))) # 카테고리 선택
    # category = driver.find_element(By.CSS_SELECTOR, c)
    cat = category.text
    print(cat)
    cat = category.text.split()[1] # 실제 카테고리 ex) 헌법
    num = laws_per_cat[cat]
    driver.execute_script("arguments[0].click();", category)

    driver.switch_to.frame('indexFrame') # html안에 html로 스위치
    mode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#selectLangKor"))) # 이건 한국어, 영어 법령으로 보겠다는 의미
    driver.execute_script("arguments[0].click();", mode)

    for p in pages:
      for ind in table_list:
        law_name = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "",  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ind.split(' > p')[0]))).text).strip()
        driver.find_element(By.CSS_SELECTOR, ind).send_keys(Keys.ENTER) # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드

        driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
        driver.switch_to.frame('lawViewTitle') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능

        try:
          haha = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lawtit > div:nth-child(4)"))).text
        except:
          boryu[cat].append(law_name)

        if haha == 'article Contents':
          boryu[cat].append(law_name)
          print(law_name)

        print(f'a number of crawled laws : {cur_law}')
        cur_law += 1
        total_law += 1
        driver.close() # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
        driver.switch_to.frame('indexFrame') # html안에 html로 스위치
        if cur_law > num:
          break
      if cur_law > num:
        break
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, p))).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
      # driver.find_element(By.CSS_SELECTOR, p).click()
      cur_sum_pages += 1
      print(f'a number of crawled pages : {cur_sum_pages}')

제1편 헌법 (31)
a number of crawled laws : 1
a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5
a number of crawled laws : 6
a number of crawled laws : 7
a number of crawled laws : 8
a number of crawled laws : 9
a number of crawled laws : 10
a number of crawled laws : 11
a number of crawled laws : 12
a number of crawled laws : 13
a number of crawled laws : 14
a number of crawled laws : 15
대한민국헌법
a number of crawled laws : 16
a number of crawled laws : 17
a number of crawled laws : 18
a number of crawled laws : 19
a number of crawled laws : 20
a number of crawled pages : 2
a number of crawled laws : 21
a number of crawled laws : 22
a number of crawled laws : 23
a number of crawled laws : 24
연호에 관한 법률
a number of crawled laws : 25
a number of crawled laws : 26
a number of crawled laws : 27
a number of crawled laws : 28
a number of crawled laws : 29
a number of crawled laws : 30
a number of crawled laws : 31
제2편 국회 (10)
a number

In [ ]:
boryu

defaultdict(list,
            {'헌법': ['대한민국헌법', '연호에 관한 법률'],
             '민사법': ['외국인의 서명날인에 관한 법률', '인지 첩부첨부 및 공탁 제공에 관한 특례법'],
             '형사법': ['비영리법인의 임원 처벌에 관한 법률'],
             '과학ㆍ기술': ['표준시에 관한 법률'],
             '재정ㆍ경제일반': ['경품류제공에관한불공정거래행위의유형및기준지정고시\n                   이 영문법령은 공정거래위원회에서 제공하였습니다',
              '과징금부과 세부기준 등에 관한 고시\n          이 영문법령은 공정거래위원회에서 제공하였습니다',
              '기술자료 제공 요구유용행위 심사지침\n           이 영문법령은 공정거래위원회에서 제공하였습니다',
              '기업결합 심사기준\n     이 영문법령은 공정거래위원회에서 제공하였습니다',
              '법령 등의 경쟁제한사항 심사지침\n         이 영문법령은 공정거래위원회에서 제공하였습니다',
              '부당특약 심사지침\n       이 영문법령은 공정거래위원회에서 제공하였습니다',
              '용역위탁 중 역무의 범위 고시\n           이 영문법령은 공정거래위원회에서 제공하였습니다',
              '중요한 표시광고사항 고시\n            이 영문법령은 공정거래위원회에서 제공하였습니다',
              '표시광고의 공정화에 관한 법률 위반사업자등에 대한 과징금부과 세부기준 등에 관한 고시\n                         이 영문법령은 공정거래위원회에서 제공하였습니다',
              '하도급거래공정화지침\n     이 영문법령은 공정거래위원회에서 제공하였습니다'],
             '관세': ['특정국가와

In [ ]:
# import json

# with open('/content/drive/MyDrive/KB/boryu.json', 'w') as f:
#   json.dump(bbb, f)


In [ ]:
bbb = {}
for k, v in boryu.items():
  temp_v = []
  for idx in range(len(v)):
    new_v = v[idx][0]
    temp_v.append(new_v)
  bbb[k] = temp_v
bbb

{'헌법': ['대한민국헌법', '연호에 관한 법률'],
 '민사법': ['외국인의 서명날인에 관한 법률', '인지 첩부첨부 및 공탁 제공에 관한 특례법'],
 '형사법': ['비영리법인의 임원 처벌에 관한 법률'],
 '과학ㆍ기술': ['표준시에 관한 법률'],
 '재정ㆍ경제일반': ['경품류제공에관한불공정거래행위의유형및기준지정고시',
  '과징금부과 세부기준 등에 관한 고시',
  '기술자료 제공 요구유용행위 심사지침',
  '기업결합 심사기준',
  '법령 등의 경쟁제한사항 심사지침',
  '부당특약 심사지침',
  '용역위탁 중 역무의 범위 고시',
  '중요한 표시광고사항 고시',
  '표시광고의 공정화에 관한 법률 위반사업자등에 대한 과징금부과 세부기준 등에 관한 고시',
  '하도급거래공정화지침'],
 '관세': ['특정국가와의 관세협상에 따른 국제협력관세의 적용에 관한 규정'],
 '상업ㆍ무역ㆍ공업': ['대규모 유통업 분야의 특약매입거래에 관한 부당성 심사지침',
  '벤처투자 촉진에 관한 법률 시행규칙',
  '선불식 할부거래에서의 소비자보호 지침',
  '전자상거래 등에서의 소비자보호 지침',
  '전자상거래 소비자보호법 위반사업자에 대한 과징금 부과기준 고시'],
 '공업소유권': ['국가핵심기술'],
 '사회복지': ['장애인복지법 시행규칙'],
 '노동': ['근로자의 날 제정에 관한 법률'],
 '해운': ['선원법 시행규칙', '선원의안전및위생에관한규칙']}

In [ ]:
total = 0
for k, v in bbb.items():
  total += len(v)
total

27

In [ ]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")

laws_per_cat = {}
# for i in range(2, 47):
for i in range(2, 47):
  category = driver.find_element(By.CSS_SELECTOR, f"#tree_{i}_span") # 카테고리 선택
  cat = category.text
  laws_per_cat[cat.split()[1]] = int(cat.split('(')[-1].replace(')',''))

# category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]

cur_sum_pages = 1
total_law = 1

while total_law <= 2341:
  for c in category_list:
    cur_law = 1
    final_df = pd.DataFrame(columns=['cat','name','eng','kor'])

    driver.switch_to.default_content()
    category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, c))) # 카테고리 선택
    # category = driver.find_element(By.CSS_SELECTOR, c)
    print(category.text)
    cat = category.text.split()[1] # 실제 카테고리 ex) 헌법
    num = laws_per_cat[cat]
    driver.execute_script("arguments[0].click();", category)

    driver.switch_to.frame('indexFrame') # html안에 html로 스위치
    mode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#selectLangKor"))) # 이건 한국어, 영어 법령으로 보겠다는 의미
    driver.execute_script("arguments[0].click();", mode)

    for p in pages:
      for ind in table_list:
        law_name = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "",  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ind.split(' > p')[0]))).text).strip()
        driver.find_element(By.CSS_SELECTOR, ind).send_keys(Keys.ENTER) # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드

        driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
        driver.switch_to.frame('lawViewContent') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능

        # req = driver.page_source
        # soup = BeautifulSoup(req, 'html.parser')
        # corpus = soup.find_all(['span', 'a']) # 내용만 긁자

        hang = driver.find_elements(By.CLASS_NAME, "hang")
        # ho = driver.find_elements(By.CLASS_NAME, "ho")
        # none = driver.find_elements(By.CLASS_NAME, "none")
        corpus = hang

        # eng = ''
        # kor = ''
        eng = []
        kor = []
        eng_num = 0
        kor_num = 0
        for i in corpus: # corpus에서 tag 하나씩 eng, kor 매칭해서 저장하기
          clean_text = i.text
          if clean_text.startswith('['):
            continue
          if len(re.sub('[^ㄱ-ㅣ가-힣]','', clean_text)) == 0:
            if len(eng) != 0 and len(kor) != 0 and (eng_num == kor_num):
              # final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'cat': cat, 'name': law_name, 'eng': eng.strip(), 'kor': kor.strip()}])])
              final_df = pd.concat([final_df, pd.DataFrame(zip([cat]*len(eng), [law_name]*len(eng), eng, kor), columns=final_df.columns)])
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            elif len(eng) != 0 and len(kor) != 0 and (eng_num != kor_num):
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            # eng += clean_text
            eng.append(clean_text)
            eng_num += 1
          else:
            if eng_num == 0:
              kor = []
              kor_num = 0
              continue
            # kor += clean_text
            kor.append(clean_text)
            kor_num += 1

        print(f'a number of crawled laws : {cur_law}')
        cur_law += 1
        total_law += 1
        driver.close() # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
        driver.switch_to.frame('indexFrame') # html안에 html로 스위치
        if cur_law > num:
          break
      if cur_law > num:
        break
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, p))).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
      # driver.find_element(By.CSS_SELECTOR, p).click()
      cur_sum_pages += 1
      print(f'a number of crawled pages : {cur_sum_pages}')

    final_df = final_df.reset_index(drop=True)
    final_df.to_csv(f'/content/drive/MyDrive/KB/hang/{cat}.csv')

제1편 헌법 (31)
a number of crawled laws : 1
a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5
a number of crawled laws : 6
a number of crawled laws : 7
a number of crawled laws : 8
a number of crawled laws : 9
a number of crawled laws : 10
a number of crawled laws : 11
a number of crawled laws : 12
a number of crawled laws : 13
a number of crawled laws : 14
a number of crawled laws : 15
a number of crawled laws : 16
a number of crawled laws : 17
a number of crawled laws : 18
a number of crawled laws : 19
a number of crawled laws : 20
a number of crawled pages : 2
a number of crawled laws : 21
a number of crawled laws : 22
a number of crawled laws : 23
a number of crawled laws : 24
a number of crawled laws : 25
a number of crawled laws : 26
a number of crawled laws : 27
a number of crawled laws : 28
a number of crawled laws : 29
a number of crawled laws : 30
a number of crawled laws : 31
제2편 국회 (10)
a number of crawled laws 

KeyboardInterrupt: ignored

1. (Enforcement Date),

과학기술1, 국가공무원1, 국가보훈2, 농업1, 수자원토지건설3, 지방제도1, 헌법1

2. 상업ㆍ무역ㆍ공업,"선불식 할부거래에서의 소비자보호 지침

이 놈 이상함 (tree구조 이상한 법령들 모두 포함)

3. 민사법

삭제 [시행일 : 2022. 7. 1.] 제6조 이런놈 잇음

입양특례법에 <<~>> 이놈 있음

4. 상업 무역공업

Articles 55-4 through 55- 이딴놈잇음

5. 수자원 토지건설

(2) ) <- 이렇게 시작하는놈 있음

6. 선거 정당

(3) ) <- 이렇게 시작하는놈 있음

7. 내국세 - 세무사법

. 로 시작하는 영어문장 있는데 . 만 지우면 의미는 동일함

선거 정당 - 공직선거법도 동일한 놈 있음

한놈은 , 로 시작하는 놈 있음 페어는 맞음 (행정일반 - 사회적 참사의 진상규명 및 안전사회 건설 등을 위한 특별법 시행령)

8. 동글뱅이 숫자가 한 본문에 두개 이상인 애들도 있고, 영어에 포함되어 있는 애들도 있음. 이런애들 다 버림.

9. 한글에서 본문의 내용이 title로 일부 가버린 애들이 있음 이런애들 다 버림

10. 영-한 문장끼리 처음 숫자는 매칭되나 내용이 일부 날아간 경우가 있음 (ex: 환경- 진동관리법 시행규칙 - the first measureing station.

11. 꺽쇠가 시작 꺽쇠는 없고 닫는 꺽쇠만 있음 ;; 재정경제-국가를 당사자로 ~ 시행령 - (7) The calculation sheet specified in paragraph
여는건 하난데 닫는건 두개인 놈도 있음

12. 특이한 꺽쇠 발견 <> ＜＞ 〈〉

13. 영어 꺽쇠 온갖 종류들 확인 완료, 한국어 꺽쇠들 확인 완료

14. 노동- 노동위원회법 시행령 - [This Article Wholly Amended by Presidential Decree No. 26420, Jul. 20, 2015]

15. hang 클래스에 이상한 애들까지 들어와서 영-한 길이가 엄청 다른것들

해운- 선박직원법 - A person who has obtained a license under the ~

행정일반- 정부출연연구기관 등의 설립운영 및 육성에 관한 법률 - Research institutes listed on the left column of the following chart at the time of the en ~

위와 동일한 법령 - Research institutes listed on the left column of the following chart (hereinafter in this paragraph referred ~

노동 - 국가기술자격법 시행령 - Notwithstanding Article 10 (2) and attached Ta ~

16. 괴상한 온점 ．

17. 지방제도- '규제자유특구 및 지역특화발전특구에 관한 규제특례법 시행령 에 lf가 숨겨져있음 (엘에프)

18. 괴상하게 his or herhis or her 들어가있는게 많음 특히 환경 - 환경분야 시험검사 등에 관한 법률

19. 내국세에 페어안맞는애 많음

20. 가운데 점 두종류 ㆍ ·

In [ ]:
import pandas as pd
import re
import os

total_num = 0
num_start = 0
## 한 paragraph을 문장단위로 쪼개기
for l in os.listdir('./new_file/'):
# for l in ['상업ㆍ무역ㆍ공업.csv']:
    df = pd.read_csv(f'./new_file/{l}', index_col=0)
    print(f'현재 카테고리: {l[:-4]}')

    final_df = pd.DataFrame(columns=df.columns)
    error = 0
    count = 0

    strange_str_count = 0
    for row in df.values:
        temp = 0
        count += 1
        category = row[0]
        law_name = row[1]

        if pd.isna(row[2]) or pd.isna(row[3]):
            # print(category, law_name)
            # print(row[2], row[3])
            continue
        if 'Omitted' in row[2] or 'Deleted' in row[2]:
            strange_str_count += 1
            continue
        if '?' in row[2] or '?' in row[3]:
            strange_str_count += 1
            continue
        if '···' in row[2] or '···' in row[3]:
            strange_str_count += 1
            continue
        if '●' in row[2] or '●' in row[3]:
            strange_str_count += 1
            continue
        if '->' in row[2] or '->' in row[3]:
            strange_str_count += 1
            continue

        for i in ['①','②','③','④','⑤','⑥','⑦','⑧','⑨','⑩','⑪','⑫','⑬','⑭','⑮']:
            temp += row[3].count(i)

        if temp > 1:
            strange_str_count += 1
            continue

        if row[2].startswith('('):
            eng = row[2][4:].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('o ').lstrip('○ ').lstrip('=> ').lstrip('⇒ ').lstrip('- ').replace('“', '"').replace('”', '"').replace('’', "'").replace('... ', "").replace('…', '') # replace(':', '.'). .replace('( )', '') .replace('  ', ' ')
        elif row[2][0] in ['0','1','2','3','4','5','6','7','8','9']:
            num_start += 1
            continue
        else:
            eng = row[2].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('o ').lstrip('○ ').lstrip('=> ').lstrip('⇒ ').lstrip('- ').replace('“', '"').replace('”', '"').replace('’', "'").replace('... ', "").replace('…', '') # .replace(':', '.')
        kor = row[3].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('- ').lstrip('⇒ ').lstrip('o ').lstrip('○ ').lstrip('=> ').replace('“', '"').replace('”', '"').replace(' · ', ', ').replace('ㆍ', ', ').replace('「', '"').replace('」', '"').replace('·',', ').replace('’', "'").replace('... ', "").replace('…', '') #
        eng = re.sub(r'\<[^>]+\>', '', eng).strip()
        kor = re.sub(r'\<[^>]+\>', '', kor).strip()

        eng = re.sub(r'\〈[^〉]+\〉', '', eng).strip()
        kor = re.sub(r'\〈[^〉]+\〉', '', kor).strip()

        eng = re.sub(r'\([\s]*\)', '', eng).strip()
        kor = re.sub(r'\([\s]*\)', '', kor).strip()

        eng = re.sub(r'[\s]{2,}', ' ', eng).strip()
        kor = re.sub(r'[\s]{2,}', ' ', kor).strip()

        kor = re.sub('[①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮].','', kor).strip()

        if '전항' in kor:
            temp_eng = re.sub(r'paragraph\s\([0-9]\)', 'ppppap', eng)
            if 'ppppap' in temp_eng:
                strange_str_count += 1
                continue
        if len(kor) == 0 or len(eng) == 0:
            strange_str_count += 1
            continue
        # split_kor = re.split(r'[^a-zA-Z0-9ㄱ-ㅣ가-힣一-龥,.:")\]”]\s', kor)
        # split_kor = [i.strip() for i in split_kor]
        # if len(split_kor) != 1:
            # split_kor.pop(0)
        # if len(split_kor) > 2:
            # import pdb;pdb.set_trace()
        # kor = split_kor[0].strip()

        # if len(eng) < 10 or len(kor) < 5:
        #     print(eng, kor)
        #     continue

        # split_eng = re.split(r'\.\([0-9]+\)\s', eng)

        # split_eng = [i.strip() for i in split_eng]


        # split_eng.pop(0)


        # if len(split_eng)!= len(split_kor):
        #     error += 1
        #     continue



        # cat = [category] * len(split_eng)
        # ln = [law_name] * len(split_eng)
        final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'cat': category, 'name': law_name, 'eng': eng, 'kor': kor}])])
    # print(f'이상한 pair 개수: {error}')
    print(f'데이터 수: {final_df.shape[0]}')

    final_df.drop_duplicates(['eng'], inplace=True)
    final_df.drop_duplicates(['kor'], inplace=True)

    total_num += final_df.shape[0]

    final_df.to_csv(f'./preprocessed/{l[:-4]}_pre.csv', index=False)
print(strange_str_count)
print(total_num)
print(num_start)

# 번역 데이터수집

In [ ]:
# 필요한 라이브러리 설치
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument('--headless') # 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# 드라이버 설정

driver = webdriver.Chrome(options=chrome_options)

# # 테스트 URL
# driver.get("http://www.google.com")


# # 타이틀 출력으로 정상 작동 확인
# print(driver.title)

# # 드라이버 종료
# driver.quit()


In [ ]:
## 번역문장 크롤링
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import os
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(options=chrome_options)

driver.implicitly_wait(10)
driver.get("https://www.technologyreview.com/topic/space/")

#### 이벤트 팝업
# button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#popmake-26048 > button')))
# driver.execute_script("arguments[0].click();", button)

#### kor 페이지 더보기 계속 누르기
# xpath = '//*[@id="content"]/div/div/div[3]/div[2]/span' # kor
xpath = '//*[@id="content-list__load-more-btn"]' # eng

count = 0
while True:
    try:
        # 버튼이 존재하는지 확인하고, 존재한다면 클릭
        load_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        driver.execute_script("arguments[0].click();", load_more_button)
        time.sleep(0.5)
        count += 1
        if count > 100:
          print('too many')
          break
    except:
        # 버튼이 없으면 루프 종료
        break

#### 특정 기사
# button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#post-25859 > div > div.col-left > h3 > a')))
# title = button.text
# driver.execute_script("arguments[0].click();", button)
# print(title)

#### 또 이벤트 팝업
# button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#popmake-26048 > button')))
# driver.execute_script("arguments[0].click();", button)

# 기사 내용 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# content = soup.select('#content--body > div p') # eng
# content = soup.select('div.single-content p') # kor

# 제목 파싱
# content = soup.select('div > div.col-left > a ') # kor 페이지에서 영어 제목들 다 파싱
# eng 페이지에서 영어 제목들 다 파싱
pattern = re.compile('teaserItem__title--.*')
content = soup.find_all(class_=pattern)

# for paragraph in content:
#     print(paragraph.text)

# driver.quit()
# driver.execute_script("arguments[0].click();", button)

# hang = driver.find_elements(By.CLASS_NAME, "p")

too many


In [ ]:
eng_title.extend([i.text for i in content])

In [ ]:
eng_title = [i.text for i in content]
len(eng_title)

793

In [ ]:
ai_title = list(set(eng_title) - (set(eng_title) - set(kor_title)))
len(ai_title)

873

In [ ]:
ai_title[:10]

['A soft e-skin mimics the way human skin can sense things',
 'Police got called to an overcrowded presentation on “rejuvenation” technology',
 'Tiny faux organs could crack the mystery of menstruation',
 'How tracking coronavirus variants will prepare us for the next global public health threat',
 'Preparing for AI-enabled cyberattacks',
 'The entrepreneur dreaming of a factory of unlimited organs',
 'How engineered microbes could cut aviation emissions',
 'Worldcoin just officially launched. Here’s why it’s already being investigated.',
 'Inside China’s unexpected quest to protect data privacy',
 'Climate change is making India’s brutal heat waves worse']

In [ ]:
import pickle

with open('/content/drive/MyDrive/KB/KB_trans/techreview/all_873_title.pkl', 'wb') as f:
  pickle.dump(ai_title, f)

In [ ]:
count = 0
for i in range(len(kor)):
  if len(kor[i].text.split('.')) != len(eng[i].text.split('.')):
    print(kor[i].text)
    print(eng[i].text)
    break

그런데 최근 구글 딥마인드가 딥러닝을 활용해 신소재 개발 과정을 획기적으로 단축하는 방법을 <네이처(Nature)>에 발표했다. 연구진은 ‘GNoME(Graphical Networks for Material Exploration)’라고 불리는 이 기술을 통해 이미 220만 개의 신소재 구조를 예측하였으며, 현재 이 중 700여 개의 후보 물질을 합성해 실험하고 있다.
Google DeepMind hopes to change that with a new tool that uses deep learning to dramatically speed up the process of discovering new materials. Called graphical networks for material exploration (GNoME), the technology has already been used to predict structures for 2.2 million new materials, of which more than 700 have gone on to be created in the lab and are now being tested. It is described in a paper published in Nature today. 


In [ ]:
eng[0]

<p>From EV batteries to solar cells to microchips, new materials can supercharge technological breakthroughs. But discovering them usually takes months or even years of trial-and-error research. </p>

In [ ]:
#content > div > div.columnArea--feed__main--e20f345f8a94d9ad1cb83d5762946a2f > ul > li:nth-child(1) > div > div.teaserItem__text--4bbad369f514cccd52cf71294fd97081 > header > h3 > a
#content > div > div.columnArea--feed__main--e20f345f8a94d9ad1cb83d5762946a2f > ul > li:nth-child(2) > div > div.teaserItem__text--4bbad369f514cccd52cf71294fd97081 > header > h3 > a